In [1]:
import numpy as np
import pymc3 as pm

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
c:\Users\oscar\AppData\Local\Programs\Python\Python311\Lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
c:\Users\oscar\AppData\Local\Programs\Python\Python311\Lib\site-packages\theano\scalar\basic.py:2323: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  self.ctor = getattr(np, o_type.dtype)


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

Funciones

In [3]:
# Define the pi_rho function
def pi_rho(tau, g, d, h, eta_ti):
    return 1 / (1 + np.exp(d * g - h * tau - eta_ti))

In [4]:
# Define the U function
def U(c, g, a, b):
    return a * np.log(c) + b * np.log(g)

# Define the V function
def V(k):
    return -np.log(k)

# Define the W function
def W(g, tau, epsilon, y, a, b, beta):
    return U(y - tau, g, a, b) + beta * V(tau + epsilon - g)


Funciones estocasticas

In [5]:
def generate_ar1(phi, sigma, length):
    # Initial value is a random normal variable
    series = [np.random.normal(0, sigma)]

    for t in range(1, length):
        series.append(phi * series[t-1] + np.random.normal(0, sigma))

    return np.array(series)

In [18]:
# Parameters for the AR(1) processes
phi_alpha = 10  # AR coefficient for alpha
sigma_alpha = 0.1  # Std dev for alpha

phi_q = 10  # AR coefficient for q
sigma_q = 0.1  # Std dev for q
length = 15  # Length of the time series

# Generate the AR(1) series
alpha_series = generate_ar1(phi_alpha, sigma_alpha, length)
q_series = generate_ar1(phi_q, sigma_q, length)

# Calculate epsilon and eta
epsilon_ti = alpha_series[1:] + alpha_series[:-1]
eta_ti = q_series[1:] + q_series[:-1]


Estimacion bayesiana de los valores de Omega

In [20]:
# Prior assumptions
phi_alpha_prior = 10
sigma_alpha_prior = 50

# Generate observed data
length = 100  # example length
alpha_series = generate_ar1(phi_alpha_prior, sigma_alpha_prior, length)
epsilon_ti_observed = alpha_series[1:] + alpha_series[:-1]

# Bayesian Model
with pm.Model() as model:
    # Priors
    phi_alpha = pm.Normal('phi_alpha', mu=10, sigma=2)
    sigma_alpha = pm.HalfNormal('sigma_alpha', sigma=0.1)
    
    # Likelihood
    epsilon_ti = pm.AR1('epsilon_ti', k=phi_alpha, tau_e=sigma_alpha**-2, observed=epsilon_ti_observed)

    # Perform inference
    trace = pm.sample(1000, return_inferencedata=True)



AttributeError: module 'pymc' has no attribute 'AR1'

In [ ]:
# Posterior analysis and calculating Omega_i and Omega_O
# ... (This part requires additional details about how to maximize W and calculate Omega)

def optimize_W(epsilon, y, a, b, beta):
    # Define the objective function to minimize (negative of W)
    def objective1(vars):
        g, tau = vars
        return -W(g, tau, epsilon, y, a, b, beta)

    # Define constraints
    constraints = [
        {'type': 'ineq', 'fun': lambda vars: vars[0]},  # g >= 0
        {'type': 'ineq', 'fun': lambda vars: y - vars[1]},  # y - tau >= 0
        {'type': 'ineq', 'fun': lambda vars: vars[1] + epsilon - vars[0]}  # tau + epsilon - g >= 0
    ]

    # Initial guess
    initial_guess = [1, 1]  # Adjust as needed

    # Perform optimization
    result = minimize(objective1, initial_guess, constraints=constraints)

    if result.success:
        return -result.fun  # Return the maximized value of W
    else:
        raise ValueError("Optimization failed")

In [ ]:
import numpy as np

def simulate_conditioned_epsilons(alpha_t_i, phi_alpha_posterior, sigma_alpha_posterior, num_simulations):

    simulated_epsilons = []

    for _ in range(num_simulations):
        # Randomly select a sample from the posterior distributions
        phi_alpha_sample = np.random.choice(phi_alpha_posterior)
        sigma_alpha_sample = np.random.choice(sigma_alpha_posterior)

        # Simulate epsilon_t+1 using the AR(1) process
        epsilon_t1 = phi_alpha_sample * alpha_t_i + np.random.normal(0, sigma_alpha_sample)
        simulated_epsilons.append(epsilon_t1)

    return np.array(simulated_epsilons)

# Example usage
alpha_t_i = ...  # Set the current value of alpha_t to alpha_i
phi_alpha_posterior_samples = ...  # Array of posterior samples for phi_alpha
sigma_alpha_posterior_samples = ...  # Array of posterior samples for sigma_alpha
num_simulations = 10000  # Number of simulations

simulated_epsilons_i = simulate_conditioned_epsilons(alpha_t_i, phi_alpha_posterior_samples, sigma_alpha_posterior_samples, num_simulations)

# Assuming y, a, b, beta are defined
# Simulate future values of epsilon from the posterior distributions

# For Omega_i
Omega_i_values = []
for epsilon in simulated_epsilons_i:  # Simulate epsilon values conditioned on alpha_t = alpha_i
    Omega_i_values.append(optimize_W(epsilon, y, a, b, beta))

Omega_i = np.mean(Omega_i_values)

# For Omega_O
Omega_O_values = []
for epsilon in simulated_epsilons_o:  # Simulate epsilon values unconditionally
    Omega_O_values.append(optimize_W(epsilon, y, a, b, beta))

Omega_O = np.mean(Omega_O_values)


________________

Estimacion de montecarlo no bayesiana

In [ ]:
def simulate_omega(W_function, epsilon_distributions, num_simulations=10000):
    """
    Simulate Omega value using Monte Carlo Simulation.
    :param W_function: The W function to maximize.
    :param epsilon_distributions: A function that generates realizations of epsilon.
    :param num_simulations: Number of simulations to run.
    :return: Estimated Omega value.
    """
    W_star_values = []

    for _ in range(num_simulations):
        epsilon_realization = epsilon_distributions()
        # Solve the maximization problem for W with the given epsilon
        result = maximize_W(W_function, epsilon_realization)
        if result.success:
            W_star_values.append(result.fun)  # Assuming maximization

    return np.mean(W_star_values)

# Example usage
# You need to define `maximize_W` function and `epsilon_distribution`
Omega_i = simulate_omega(W, epsilon_distribution_for_Omega_i)
Omega_O = simulate_omega(W, epsilon_distribution_for_Omega_O)


____________________

In [ ]:
# Parameters
beta = ...
X = ...
epsilon_i = ...

# Objective Function
def objective(vars, epsilon_ti, eta_ti):
    g, tau = vars
    pi_rho_value = pi_rho(tau, g, d, h, eta_ti)
    W_value = W(g, tau, epsilon_ti, y, a, b, beta)
    return -1 * (beta * (X * (1 + beta) + Omega_i - Omega_O) * pi_rho_value + W_value)


# Constraints
constraints = [
    {'type': 'ineq', 'fun': lambda vars: vars[0]},  # g >= 0
    {'type': 'ineq', 'fun': lambda vars: vars[1]},  # tau >= 0
    {'type': 'ineq', 'fun': lambda vars: vars[1] + epsilon_i - vars[0]}  # tau + epsilon_i - g >= 0
]

In [ ]:
# Initial guess
initial_guess = [1, 1]  # Adjust as needed

# Example time period t
t = ...  # specify the time period

# Solve the problem for time period t
result = minimize(lambda vars: objective(vars, epsilon_ti[t], eta_ti[t]), initial_guess, constraints=constraints)

# Output the result for time period t
if result.success:
    optimized_g, optimized_tau = result.x
    print(f"Optimized g: {optimized_g}, Optimized tau: {optimized_tau} for time period {t}")
else:
    print("Optimization failed for time period {t}.")